# Классификация дефектов кожи

## Дипломная работа Жеманцевой Татьяны. Skillfactory.

## Задача: создание модели классификации, которая по представленной картинке кожного покрова сможет определить варианты с процентным 
## соотношением похожести данного дефекта на известное заболевание.
 
## Данные взяты с kaggle - закрытое соревнование, данные для которого взяты и отсортированы в ISIC (https://www.isic-archive.com).
## В данной работе выполнена классификация с использованием CNN и предобученой нейросети в Tensorflow - Xception.

In [27]:
#!pip install -q tensorflow==2.3
#!pip install keras_efficientnets
#!pip freeze > requirements.txt

# Импортируем библиотеки 

In [28]:
# Импортируем библиотеки

import pandas as pd # импортируем необходимые библиотеки
import numpy as np
import re
import random
import os
import sys
import PIL
import cv2

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import ExtraTreeRegressor
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler 

from collections import Counter
from sklearn.metrics import confusion_matrix, classification_report
import itertools 

#from catboost import CatBoostRegressor

from tqdm import tqdm
from tqdm.notebook import tqdm

from itertools import combinations
from scipy.stats import ttest_ind


# # keras
import tensorflow as tf
from tensorflow import keras
#import tensorflow.keras.layers as L
#from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#import albumentations
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
#from tensorflow.keras.layers import *
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
#import tensorflow as tf
import tensorflow.keras 
import tensorflow.keras.models #as M
#import tensorflow.keras.layers as L
#import tensorflow.keras.backend #as K
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay as ExpDecay

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from glob import glob
from glob import iglob
#from keras.applications import InceptionV3
#from keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense ,LeakyReLU,UpSampling2D, GlobalAveragePooling2D,Conv2DTranspose, BatchNormalization,GlobalMaxPool2D, Convolution2D

import nltk
from nltk.corpus import stopwords

# plt
import matplotlib.pyplot as plt
import seaborn as sns
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
#DATA_PATH = "/content/drive/My Drive/cars2/"
#PATH = "/content/"  # рабочая директория

In [29]:
#from tensorflow import keras
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications import EfficientNetB4
print(keras.__version__)
print(tf.__version__)

2.4.0
2.4.0


In [30]:
print(tf.__version__)

2.4.0


In [31]:
#from tensorflow.keras.applications import InceptionV3#

In [32]:
#keras.applications.EfficientNetB4()

In [33]:
print('Python       :', sys.version.split('\n')[0])
print('Numpmodel.summary()y        :', np.__version__)
print('Tensorflow   :', tf.__version__)

Python       : 3.7.9 | packaged by conda-forge | (default, Dec  9 2020, 21:08:20) 
Numpmodel.summary()y        : 1.19.5
Tensorflow   : 2.4.0


# Задаем константы

In [34]:
train_path ='/kaggle/input/dermnet/train'
test_path = '/kaggle/input/dermnet/test'
#train_path = "/kaggle/input/skin-cancer9-classesisic/skin cancer isic the international skin imaging collaboration/Skin cancer ISIC The International Skin Imaging Collaboration/Train/"
#test_path = "/kaggle/input/skin-cancer9-classesisic/skin cancer isic the international skin imaging collaboration/Skin cancer ISIC The International Skin Imaging Collaboration/Test/"

In [35]:
RANDOM_SEED = 42
IMAGE_SIZE         = (299, 299)
#INPUT_SHAPE        = (224, 224, 3)
IMG_SIZE           = 299 # размер входного изображения для Xception по-умолчанию
IMG_CHANNELS       = 3   # у RGB 3 канала
input_shape        = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)
# Keras settings
BATCH_SIZE = 24
VAL_BATCH_SIZE = 24
VAL_SPLIT          = 0.1 # сколько данных выделяем на тест = 15%
EPOCHS = 12
LR = 0.00001

# Предобработка изображений: просмотр, аугментация, создание обучающего и тестового наборов

In [ ]:
# Посмотрим пример картинки

img = PIL.Image.open(train_path + "nevus/ISIC_0000041.jpg") # load_img(train_path + "nevus/ISIC_0000041.jpg")  "ISIC_0000398.jpg"
plt.imshow(img)
plt.axis("off")
plt.show()


In [ ]:
# Преобразуем картинку в массив

x = img_to_array(img)
print(x.shape)

In [36]:
# C помощью glob определяем сколько разных папок-классов с картинками имеется.

className = glob(train_path + '/*' )
numberOfClass = len(className)
print("NumberOfClass: ",numberOfClass)

NumberOfClass:  23


In [ ]:
className

## Аугментация с помощью ImageDataGenerator (выполняется либо этот раздел аугментации, либо следующий )

In [11]:
# Аугментация обучающего датасета
# Официальная документация: https://keras.io/preprocessing/image/

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range = 60,
    brightness_range = [0.5, 1.5],
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=VAL_SPLIT, # set validation split
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=[0.75,1.25],
    fill_mode='reflect')  #  fill_mode='nearest'

       
test_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255, validation_split=VAL_SPLIT)

In [ ]:
# Завернем наши данные в генератор.

train_generator = train_datagen.flow_from_directory(
    train_path,      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, 
    seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = val_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, 
    seed=RANDOM_SEED,
    subset='validation') # set as validation data


In [ ]:
test_val_generator = test_datagen.flow_from_directory(
    directory=test_path,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE)


In [ ]:
train_generator

## Аугментация с помощью albumentations (выполняется либо этот раздел аугментации, либо предыдущий)

In [12]:
!pip install albumentations

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [13]:
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

  Cloning https://github.com/mjkvaak/ImageDataAugmentor to /tmp/pip-req-build-1vuwvk5v
  Running command git clone -q https://github.com/mjkvaak/ImageDataAugmentor /tmp/pip-req-build-1vuwvk5v
  Created wheel for ImageDataAugmentor: filename=ImageDataAugmentor-0.0.0-py3-none-any.whl size=29531 sha256=2c6a20155994ad4e0b2d52e6ddd302a025fcd8017be31b6f73cf5c9d52da048f
  Stored in directory: /tmp/pip-ephem-wheel-cache-m_oas689/wheels/c9/bd/73/9cfa59d2393dae55bbcc30f5aa901f55fe531c66efebbc8fc3
Successfully built ImageDataAugmentor
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [37]:
#Аугментация данных важна, когда мы работаем с небольшим датасетом. Это как раз наш случай.

import albumentations
import cv2
from ImageDataAugmentor.image_data_augmentor import *
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)

In [ ]:
# попробовать этот вариант

AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5), #0.5
    albumentations.Rotate(limit=90),
    albumentations.JpegCompression(quality_lower=85, quality_upper=100, p=0.5),
    albumentations.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
   # albumentations.HueSaturationValue(p=0.5),
    albumentations.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.2, hue=0.00, always_apply=False, p=0.5),
    albumentations.RandomContrast(limit=0.1, p=0.5),
    albumentations.Resize(IMG_SIZE, IMG_SIZE)
])

train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = AUGMENTATIONS,
        validation_split=VAL_SPLIT,
        )

      
test_datagen = ImageDataAugmentor(rescale=1./255)

val_datagen = ImageDataAugmentor(rescale=1. / 255, validation_split=VAL_SPLIT)

In [38]:
# Либо этот вариант. 


AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5), #0.5
    albumentations.Rotate(limit=20, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
    albumentations.OneOf([
        albumentations.CenterCrop(height=IMG_SIZE, width=IMG_SIZE),
        albumentations.CenterCrop(height=IMG_SIZE, width=IMG_SIZE),
    ],p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=0.5),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
   # albumentations.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
    albumentations.Resize(IMG_SIZE, IMG_SIZE)
])

train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = AUGMENTATIONS,
        validation_split=VAL_SPLIT,
        )

      
test_datagen = ImageDataAugmentor(rescale=1./255)

val_datagen = ImageDataAugmentor(rescale=1. / 255, validation_split=VAL_SPLIT)

In [39]:
# Завернем наши данные в генератор:

train_generator = train_datagen.flow_from_directory(
    train_path,      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = val_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data



Found 14011 images belonging to 23 classes.
Found 1546 images belonging to 23 classes.


In [40]:
test_val_generator = test_datagen.flow_from_directory(
    directory=test_path,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE)

Found 4002 images belonging to 23 classes.


In [ ]:
# Переведем картинки в массивы

train_pairs = [next(train_generator) for i in range(len(train_generator))]
val_pairs = [next(test_generator) for i in range(len(test_generator))]
train_images = np.concatenate([x[0] for x in train_pairs], axis = 0)
train_labels = np.concatenate([x[1] for x in train_pairs], axis = 0)
val_images = np.concatenate([x[0] for x in val_pairs], axis = 0)
val_labels = np.concatenate([x[1] for x in val_pairs], axis = 0)

In [ ]:
#val_df=val_labels.flatten() 
tr_df = train_labels.flatten() 

In [ ]:
# Прсмотрим несколько картинок после применения аугментации.

from skimage import io

def imshow(image_RGB):
    io.imshow(image_RGB)
    io.show()

x,y = train_generator.next()
print('Пример картинок из train_generator')
plt.figure(figsize=(12,8))

for i in range(0,6):
    image = x[i]
    plt.subplot(3,3, i+1)
    plt.imshow(image)
   # plt.title('Class: '+str(y[i]))
    #plt.axis('off')
plt.show()

In [ ]:
imgs, labels = next(train_generator)
print(imgs.shape, labels.shape)

In [ ]:
imgs, labels = next(test_generator)
print(imgs.shape, labels.shape)
print(len(test_generator))

# Посмотрим распределение данных по классам.

In [ ]:

def classes_count (path):
    # определяет содержимое папок классов
    
    classes = []
    for filename in iglob(os.path.join(path, "**","*.jpg")):
        classes.append(os.path.split(os.path.split(filename)[0])[-1])
    cnt = Counter(classes)
    return cnt

def percentage_value(pct, allvals):
    # вычисляет процентное значение размера класса
    
    absolute = int(pct/100.*np.sum(allvals))
    return "{:.1f}%\n({:d})".format(pct, absolute)

def plot_dataset_description(path, title):
    # строит круговую диаграмму
    
    classes_cnt = classes_count(path)
    values = list(classes_cnt.values())
    labels = list(classes_cnt.keys())

    plt.figure(figsize=(8,8))
    plt.pie(values, labels=labels, autopct=lambda pct: percentage_value(pct, values), 
            shadow=True, startangle=140)

    plt.title(title + ' размера ' + str(sum(classes_cnt.values())))    
    plt.show()

In [ ]:
plot_dataset_description(os.path.join(train_path), "Содержание обучающей выборки")

In [ ]:
plot_dataset_description(os.path.join(test_path), "Содержание тестовой выборки")

In [ ]:
def plot_dataset_description_bar(path, title):
    # строит диаграмму столбиками
 
    classes_val = classes_count(path)
    

    values = list(classes_val.values())
    labels = list(classes_val.keys())
    #print (labels)
    #index = [n for n in range(numberOfClass)]
    plt.figure(figsize=(10, 5))
    #hist, bins = np.histogram(values, bins=9)
    plt.bar(values, (values), width=10)
 #   plt.bar(range(len(values)),[values[k] for k in values], width=20)
    
    #plt.xlabel('Градация болезней', fontsize=10)
    plt.ylabel('Количество картинок', fontsize=10)
    plt.xticks(values, labels, fontsize=8, rotation=90)
    plt.title(title)

In [ ]:
plot_dataset_description_bar(os.path.join(train_path), 'Содержание примеров в обучающей выборке')

In [ ]:
plot_dataset_description_bar(os.path.join(test_path), 'Содержание примеров в тестовой выборке')

# Анализ данных показывает, что наша выборка несбалансирована, поскольку самый многочисленный класс в 5 раз превышает самый маленький.
# В этом случае нам подошли бы метрики precision, recall, F1-score, учитывающие веса классов, но они используются только в бинарной классификации. А проблема accuracy в том, что она не учитывает веса классов.
# Поэтому для каждого класса добавлены веса классов в параметр class_weight при обучении,что позволило повысить точность модели.
# В данной задаче, как и большинстве задач, связанных с медициной наиболее критичной является ошибка первого рода.

 
# В задаче для оценки эффективности  модели в процессе обучения я использовала
# 1. accurancy - без использования class_weight максимальный результат достигал порядка 56%
# 2. Для многоклассовой выборки с несбалансированными классами на просторах интернета рекомендуют использовать метрику macro F1. Создала свои функции для подсчета loss и macro F1 - решение давало достаточно низкую степень обучаемости, от идеи отказалась. на 45 эпохе результаты: loss: 1.0659 - f1: 0.2935 - val_loss: 1.8877 - val_f1: 0.1625
# Также попробовала оптимизацию по созданной функции потерь F1_loss - на 40й эпохе результат F1 = 0.23.

# 3. Оптимизировала модель по loss - категориальная кросс энтропия. 

 



# Построение и обучение модели.

In [ ]:
#model = keras.models.load_model('/kaggle/input/inputmod/best_model_step1.hdf5')

## Вынесем функции для раздела построения и обучения модели.

In [41]:
def info_layers():
    # вывод информации по слоям модели в процессе тюнинга
    
    print("Number of layers in the model:", len(model.layers))
    print("Number of trainable_variables layers in the model:", len(model.trainable_variables))
    print("Number of layers in the base model: ", len(base_model.layers))
    print("Number of variables layers in the base model: ", len(base_model.trainable_variables))
    print("Layers in the model:")
    for layer in model.layers:
        print(layer, layer.trainable)

def graph(history):
    # построние графиков изменения метрики и функции потерь в процессе обучения по эпохам
    
    plt.figure(figsize=(8, 4))
    plt.plot(history.history['accuracy'], 'b*-', label="training acc")
    plt.plot(history.history['val_accuracy'], 'r*-', label="validation acc")
    plt.grid()
    plt.title("train_acc vs val_acc")
    plt.ylabel("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()

    plt.figure(figsize=(8, 4))
    plt.plot(history.history['loss'], 'b*-', label="training loss")
    plt.plot(history.history['val_loss'], 'r*-', label="validation loss")
    plt.grid()
    plt.title("Training and validation loss")
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    
    


In [ ]:
# Создадим собственную метрику f1 и функцию потерь - f1_loss

from keras import backend as K

def f1(y_true, y_pred):
    # метрика macro F1
    
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    # функция потерь
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [42]:
# CallBack для сохранения модели, ее весов и лучшей модели, изменения LR в случае отсутствия прогресса обучения, и остановка обучения

weightpath = "best_model.hdf5"
#checkpoint = ModelCheckpoint(weightpath , monitor = 'val_f1' , mode = 'max', verbose = 1,save_best_only=True)
#checkpoint = ModelCheckpoint(weightpath , monitor = 'val_precision' , mode = 'max', verbose = 1,save_best_only=True) #save_weights_only=True
checkpoint = ModelCheckpoint(weightpath , monitor = 'val_accuracy' , mode = 'max', verbose = 1,save_best_only=True) #save_weights_only=True
reducer = ReduceLROnPlateau(monitor='loss', patience= 3,factor=0.2, min_lr=0.00000001,  verbose = 1, cooldown=2, mode='min',min_delta=0.0000001) #min_delta=0.000000001
early_stop= EarlyStopping(monitor='loss',patience= 10, mode='min')  #min_delta=0.000000001


In [43]:
callbacks_list = [checkpoint, reducer, early_stop] 

In [ ]:
# Выведем классы для удобства проставления весов

classes = {v: k for k, v in train_generator.class_indices.items()}
print(classes)

In [44]:
# Установим вес "2"и "3" для классов, размер которых в 2-3 раза меньше самых больших классов, сделав эти классы в 2-3 раза важнее

class_weight = {0: 1., 1: 0.6, 2: 1., 3: 1., 4: 1., 5: 0.6, 6: 1., 7: 1., 8: 1.,9: 1., 10: 1., 11: 1., 12: 0.6, 13: 1., 14: 0.6, 15: 1., 16: 0.6, 17: 1.,18: 0.6, 19: 1., 20: 1., 21: 1., 22: 1.}

## Шаг1  Сформируем модель

In [ ]:
# В процессе обучения проводились опыты с разными моделями, в качестве base_model необходимо выбрать  Xception, 
# как показавшую лучшие результаты

#base_model2 = EfficientNetB4(weights='imagenet', include_top=False, input_shape=input_shape)

In [45]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

83689472/83683744 [==============================] - 1s 0us/step


In [46]:
# Установим все слои base_model в неактивное состояние

base_model.trainable = False

In [ ]:
model=Sequential()
model.add(base_model)

#model.add(GlobalAveragePooling2D(),)
#model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.15))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
#model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))
#model.add(BatchNormalization())
print('ок')
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
#model.add(L.Dense(512,activation='relu'))
#model.add(L.BatchNormalization())
model.add(Dense(numberOfClass, activation='softmax'))

In [ ]:
# лучшая модель 

model=Sequential()
##model.add(base_model)

##model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D())
print('ок')
model.add(Flatten())
#model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
#model.add(L.Dense(512,activation='relu'))
#model.add(L.BatchNormalization())
model.add(Dense(numberOfClass, activation='softmax'))

In [ ]:
train_generator.samples

In [ ]:
test_generator.samples

In [ ]:
model = Sequential()
model.add(Xception(include_top = False, weights = "imagenet",
                        input_shape=(IMG_SIZE,IMG_SIZE, 3)))
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dense(numberOfClass, activation = "softmax"))



In [47]:
# CNN модель с добавлением Xception, которая показала наилучший результат 

model = Sequential()
 
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(numberOfClass, activation='softmax'))



In [ ]:
# Посмотрим информацию о количестве слоев модели и модели Imagenet

info_layers()

In [48]:
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 102

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
info_layers()

In [ ]:
# Посмотрим структуру модели

model.summary()

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install pydot

In [ ]:
# Посмотрим на схему модели.

from keras.utils import plot_model
plot_model(model,show_shapes=True,show_layer_names=True)

In [49]:
# Компилируем модель

#f1_m = 2*((tf.keras.metrics.Precision()*tf.keras.metrics.Recall())/(tf.keras.metrics.Precision()+tf.keras.metrics.Recall()+tf.epsilon()))
LR=0.0008
model.compile(optimizer=optimizers.Adam(LR),loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=optimizers.Adam(LR),loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision()])
#model.compile(optimizer=optimizers.Adam(LR),loss=f1_loss, metrics=f1)
#model.compile(optimizer=optimizers.Adam(LR),loss='categorical_crossentropy', metrics=f1)

In [ ]:
%%time
history = model.fit(
        train_generator,
        #train_images, train_labels,
        steps_per_epoch=len(train_generator), #train_generator.samples, # train_generator.samples # len(train_generator)
        epochs= 56, #EPOCHS,
        class_weight=class_weight,
        validation_data=test_generator,
        #validation_data= val_images,
        #validation_steps=len(test_generator), #test_generator.samples,  # test_generator.samples
        callbacks = callbacks_list)

Epoch 1/56
  6/584 [..............................] - ETA: 3:54 - loss: 2.5882 - accuracy: 0.0654

In [ ]:
history.history
#loss: 0.1100 - precision: 0.9824 - val_loss: 4.3982 - val_precision: 0.4840   -- 0.00008
#loss: 1.6093 - precision_1: 0.7621 - val_loss: 1.9980 - val_precision_1: 0.6232  -- LR=0.0007
#loss: 1.8622 - precision_2: 0.6718 - val_loss: 1.7129 - val_precision_2: 0.6943  --0.001
#loss: 1.5776 - precision_2 0.8497 - val_loss: 2.0402 - val_precision_3: 0.5302  --0.00017

In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)

model.save('model_last_9v1_23.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
#model = keras.models.load_model('model_last_111.hdf5')

In [ ]:
graph(history)

# Шаг2 Изменим количество неактивных слоев в сети imagenet и дообучим модель

In [ ]:
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 80

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
# Посмотрим информацию о количестве слоев модели 

info_layers()

In [ ]:
# Посмотрим структуру модели

model.summary()

In [ ]:
# Компилируем модель

LR=0.00075
model.compile(optimizer=optimizers.Adam(LR),loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=optimizers.Adam(LR),loss=f1_loss, metrics=f1)
#model.compile(optimizer=optimizers.Adam(LR),loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision()])

In [ ]:
#scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
#print("Accuracy: %.2f%%" % (scores[1]*100))


In [ ]:
%%time
history = model.fit(
        train_images, train_labels,
        steps_per_epoch=len(train_generator), #train_generator.samples, # train_generator.samples # len(train_generator)
        epochs= 50, #EPOCHS,
        class_weight=class_weight,
        validation_data=test_generator,
        validation_steps=len(test_generator), #test_generator.samples,  # test_generator.samples
        callbacks = callbacks_list)

In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)

model.save('model_last_v2.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
graph(history)

In [ ]:
# Шаг3
#model = keras.models.load_model('/kaggle/input/inputqw/model_last_017_83.hdf5')

In [ ]:
# Шаг3 Снова изменим количество неактивных слоев в сети imagenet и дообучим модель

In [ ]:
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 31

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False


In [ ]:
# Посмотрим информацию о количестве слоев модели 

info_layers()

In [ ]:
# Посмотрим структуру модели

model.summary()

In [ ]:
# Компилируем модель

LR=0.00095
model.compile(optimizer=optimizers.Adam(LR),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
#print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
%%time
history = model.fit(
        train_images, train_labels,
        steps_per_epoch=len(train_generator), #train_generator.samples, # train_generator.samples # len(train_generator)
        epochs= 48, #EPOCHS,
        class_weight=class_weight,
        validation_data=test_generator,
        validation_steps=len(test_generator), #test_generator.samples,  # test_generator.samples
        callbacks = callbacks_list)

In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)

model.save('model_last_v3.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
graph(history)

In [ ]:
# Шаг4

In [ ]:
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 11

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False



In [ ]:
# Посмотрим информацию о количестве слоев модели 

info_layers()

In [ ]:
# Посмотрим структуру модели

model.summary()

In [ ]:
# Компилируем модель

LR=0.00005
model.compile(optimizer=optimizers.Adam(LR),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
history = model.fit(
        train_images, train_labels,
        steps_per_epoch=train_generator.samples, #train_generator.samples, # train_generator.samples # len(train_generator)
        epochs= 45, #EPOCHS,
        class_weight=class_weight,
        validation_data=test_generator,
        validation_steps=(test_generator.samples), #test_generator.samples,  # test_generator.samples
        callbacks = callbacks_list)

In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)

model.save('model_last_19.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
graph2(history)

# Результаты

In [ ]:
# Считаем финальную точность модели по Accuracy и Loss

score_loss, score_acc = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("F1: %.2f%%" % (score_acc*100))
print("Loss: %.2f%%" % (score_loss))

## Confugn matrix по тестовым данным

In [ ]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    # функция рисования confusion_matrix

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    plt.figure(figsize=(12,12))

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=18)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=8)
    plt.yticks(tick_marks, classes, fontsize=12)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=16)
    plt.xlabel('Predicted label', fontsize=16)

In [ ]:
%%time
Y_pred = model.predict(test_generator)  # test_val_generator
Y_pred

In [ ]:
y_pred2 = np.argmax(Y_pred, axis=1)
y_pred2

In [ ]:
cnf_matrix = confusion_matrix(test_generator.classes, y_pred2)

In [ ]:
classes = {v: k for k, v in train_generator.class_indices.items()}
print(classes)

In [ ]:
list(classes.values())

In [ ]:
plot_confusion_matrix(cnf_matrix, list(classes.values()))

In [ ]:
# классификация ответов

print(classification_report(test_generator.classes, y_pred2, target_names=list(classes.values())))

In [ ]:
# Confugn matrix по валидационным данным

In [ ]:
%%time
Y_pred1 = model.predict(test_val_generator,verbose=1)  # test_val_generator
y_pred_val = np.argmax(Y_pred1, axis=1)
y_pred_val

In [ ]:
cnf_matrix_val = confusion_matrix(test_val_generator.classes, y_pred_val)
classes = {v: k for k, v in test_val_generator.class_indices.items()}
print(classes)

In [ ]:
plot_confusion_matrix(cnf_matrix_val, list(classes.values()))

In [ ]:
# классификация ответов

print(classification_report(test_val_generator.classes, y_pred_val, target_names=list(classes.values())))

# Tестовая демонстрация.

In [ ]:
def load_image(filename):
    # загружает картинку
    
    img = cv2.imread(os.path.join(train_path, filename))
    img = cv2.resize(img, (IMAGE_SIZE[0], IMAGE_SIZE[1]) )
    img = img /255
    return img

def predict(image):
    # возвращает первые три максимальных значений из предсказанных для каждого класса
    tta_step = 3
    predictions = []
    for i in range(tta_step):
        preds = model.predict(np.asarray([img]))[0]
        predictions.append(preds)
    pred = np.mean(predictions, axis=0)
    probabilities = pred
#    probabilities = model.predict(np.asarray([img]))[0]
    class_idx = (-probabilities).argsort()[:3]  # np.argmax(probabilities)
    return class_idx, probabilities[class_idx]

In [ ]:
classes = {v: k for k, v in train_generator.class_indices.items()}
print(classes)

In [ ]:
for idx, filename in enumerate(random.sample(test_generator.filenames, 3)):    # test_val_generator
    print("Источник: class: %s, file: %s" % (os.path.split(filename)[0], filename))
    
    img = load_image(filename)
    prediction = predict(img)
    print("Предсказано: класс: %s, с вероятностью: %f" % (classes[prediction[0][0]], prediction[1][0]))
    print("Предсказано: класс: %s, с вероятностью: %f" % (classes[prediction[0][1]], prediction[1][1]))
    print("Предсказано: класс: %s, с вероятностью: %f" % (classes[prediction[0][2]], prediction[1][2]))
    plt.imshow(img)
    plt.figure(idx)    
    plt.show()

# Выводы:
#     На несбалансированной выборке с небольшим набором данных удалось добиться точности за счет следующих шагов:
#         1. выбор подходящей к задаче архитектуры сети c Fine-tuning
#         2. задание весов для классов
#         3. уменьшение batch_size
#         4. подбор LR
#         5. использование аугментации ablumentation
        